Imports

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import dask.dataframe as dd
import dask.dataframe as dd
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from dask.distributed import LocalCluster, Client

Getting the csv data

In [3]:
DATA_DIR = "data/parking-violations"

In [4]:
# data directories
DATA_DIR = 'data/parking-violations/'
WEATHER_DIR = 'data/weather/'
AUGMENTED_DIR = 'data/augmented/'
SCHOOLS_DIR = 'data/schools/'
ATTRACTIONS_DIR = 'data/attractions/'
BUSINESS_DIR = 'data/businesses/'
CRIME_DIR = 'data/crime/'
EVENTS_DIR = 'data/events/'



In [5]:
# print current working directory and list files in it
print(os.getcwd())
print(os.listdir())


c:\Users\Uporabnik\Documents\FRI\BD_project\5-machine-learning
['machine-learning.ipynb', 'models_hourly.ipynb', 'm_high_ticket_days.py']


In [33]:
# a file to print results (For all three scenarios compare performance in terms of loss (error), scalability, time, and total
#memory consumption., time taken for each step,, results of ml models, etc.)
results = open("results.txt", "w")
results.write("Results\n\n")

9

In [35]:
time_dict = {}

In [36]:
n_workers = 4
memory_limit = '8GB'
cluster = LocalCluster(n_workers=n_workers, memory_limit=memory_limit)

client = Client(cluster) # make this bigger, so memory limit at least 8GB
print(client)
results.write("Client: {}\n\n".format(client))
results.write("Number of workers: {}\n".format(n_workers))
results.write("Memory limit: {}\n".format(memory_limit))
results.write("\n\n")

time_dict["n_workers"] = n_workers
time_dict["memory_limit"] = memory_limit


c:\Users\Uporabnik\.conda\envs\bdproject\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54073 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:54074' processes=4 threads=16, memory=29.80 GiB>


In [6]:
# get to c:\Users\Uporabnik\Documents\FRI\BD_project\data\parquet_files
directory_path = 'C:/Users/Uporabnik/Documents/FRI/BD_project/data/parquet_files'
os.chdir(directory_path)

In [42]:
# OPTIONAL: SUBSAMPLE PARQUET DATA
# read in the parquet files but only the first 10 partitions
start = time.time()
df = dd.read_parquet(directory_path + '/*.parquet', engine='pyarrow', gather_statistics=True, num_partitions=10)
end = time.time()
print("Time to read A SUBSAMPLE of parquet files: ", end-start)


time_dict['read_parquet'] = end - start

Time to read A SUBSAMPLE of parquet files:  0.16772675514221191


In [69]:
start = time.time()
df = dd.read_parquet(directory_path + '/*.parquet', engine='pyarrow', )
end = time.time()
print("Time to read parquet files: ", end-start)

time_dict['read_parquet'] = end - start

Time to read parquet files:  0.08959269523620605


In [70]:
# subsample the data
df = df.sample(frac=0.05, random_state=1) #.compute()

In [44]:
df.dtypes

for col in df.columns:
    print(col, df[col].head())
    

Summons Number 161877    7353165479
21723     1335819782
4451      1356329457
95029     1356474573
13231     1357424668
Name: Summons Number, dtype: string
Plate ID 161877    ETL1290
21723      XU584L
4451      86542MA
95029     EMW8664
13231     44311MD
Name: Plate ID, dtype: string
Registration State 161877    NY
21723     NJ
4451      NY
95029     NY
13231     NY
Name: Registration State, dtype: string
Plate Type 161877    PAS
21723     PAS
4451      COM
95029     PAS
13231     COM
Name: Plate Type, dtype: string
Issue Date 161877    2013-08-01
21723     2013-07-22
4451      2013-07-12
95029     2013-07-30
13231     2013-07-18
Name: Issue Date, dtype: string
Violation Code 161877    21
21723      9
4451      20
95029     40
13231     47
Name: Violation Code, dtype: string
Vehicle Body Type 161877    4DSD
21723     DELV
4451      SUBN
95029      SDN
13231     REFG
Name: Vehicle Body Type, dtype: string
Vehicle Make 161877     AUDI
21723     DODGE
4451      ROVER
95029     ME/BE
13231

In [71]:
len(df)

6483362

In [54]:
df.dtypes

Summons Number                       string[pyarrow]
Plate ID                             string[pyarrow]
Registration State                   string[pyarrow]
Plate Type                           string[pyarrow]
Issue Date                           string[pyarrow]
Violation Code                       string[pyarrow]
Vehicle Body Type                    string[pyarrow]
Vehicle Make                         string[pyarrow]
Issuing Agency                       string[pyarrow]
Street Code1                         string[pyarrow]
Street Code2                         string[pyarrow]
Street Code3                         string[pyarrow]
Vehicle Expiration Date              string[pyarrow]
Violation Location                   string[pyarrow]
Violation Precinct                   string[pyarrow]
Issuer Precinct                      string[pyarrow]
Issuer Code                          string[pyarrow]
Issuer Command                       string[pyarrow]
Issuer Squad                         string[py

### Data preparation

In [72]:
# Columns to keep
columns_to_keep = [
    'Issue Date', 
    #'Plate ID', 
    #'Violation Code', 
    #'Registration State', #- for plotting
    #'Plate Type', 
    #'Vehicle Body Type', 
    #'Vehicle Color', 
    #'Vehicle Make', 
    #'Issuing Agency', 
    'Violation Time', 
    #'Vehicle Year', 
    #'Time First Observed', 
    'Violation County', # we have alread
    #'Street'
    'PRCP'
]

In [73]:
df = df[columns_to_keep]

In [77]:
start = time.time()

df = df[columns_to_keep]

# Convert the selected columns to correct type
df['Issue Date'] = dd.to_datetime(df['Issue Date'], format="mixed", errors='coerce')
df['Violation Time'] = df['Violation Time'].str.slice(stop=-1) + ' ' + df['Violation Time'].str.slice(start=-1).replace({'A': 'AM', 'P': 'PM'})
df['Violation Time'] = dd.to_datetime(df['Violation Time'], format='%I%M %p', errors='coerce')
df["PRCP"] = dd.to_numeric(df["PRCP"], errors='coerce')

# Create columns for day, month and year
df['day'] = df['Issue Date'].dt.day
df['month'] = df['Issue Date'].dt.month
df['year'] = df['Issue Date'].dt.year
df['day_of_week'] = df['Issue Date'].dt.dayofweek

# Remove 'A' and 'P' from the end of the time, add ' AM' or ' PM' accordingly
df['violation_hour'] = df['Violation Time'].dt.hour

df = df.dropna()

# We already cleaned the County data with a dict, and cleaned the outlier dates
end = time.time()
print("Time to clean the data: ", end-start)

time_dict['clean_data'] = end - start

# Verify the changes
df.head()

Time to clean the data:  0.020247936248779297


,Issue Date,Violation Time,Violation County,PRCP,day,month,year,day_of_week,violation_hour
161877,2013-08-01,1900-01-01 11:43:00,NY,0.65,1,8,2013,3,11.0
21723,2013-07-22,1900-01-01 11:05:00,NY,0.06,22,7,2013,0,11.0
4451,2013-07-12,1900-01-01 09:50:00,NY,0.25,12,7,2013,4,9.0
13231,2013-07-18,1900-01-01 07:00:00,NY,0.0,18,7,2013,3,7.0
39493,2013-07-26,1900-01-01 17:22:00,NY,0.0,26,7,2013,4,17.0


In [83]:
df.dtypes

Issue Date           datetime64[ns]
Violation Time       datetime64[ns]
Violation County    string[pyarrow]
PRCP                          Int64
day                           int32
month                         int32
year                          int32
day_of_week                   int32
violation_hour                int32
dtype: object

In [67]:
# Drop the original columns
df = df.drop(columns=['Issue Date', 'Violation '])


#TODO: We also have to select the appropriate locations here: so all, county, or most interesting streets.

In [ ]:
df = df[df['Violation County'] == 'NY']
df

## Grouping data

In [87]:
# Exclude datetime columns from groupby
datetime_columns = ['Issue Date', 'Violation Time']
non_datetime_columns = [col for col in df.columns if col not in datetime_columns]
daily_tickets = df.groupby(non_datetime_columns).sum()



TypeError: datetime64 type does not support sum operations